In [10]:
import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import applyG_block_state, get_perms

import numpy as np
import quimb.tensor as qtn
import quimb
from quimb.tensor.tensor_2d_tebd import TEBD2D, LocalHam2D
import scipy
import h5py
import qib
from scipy.sparse.linalg import expm_multiply

Vlist = []
with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_rS1_niter15_3hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]
control_layers = [0, 4, 5, 9, 10, 14]
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/tfim2d_ccU_SPARSE_103_Lx4Ly4_t0.25_layers15_niter20_rS5_DECOMPOSE_n9_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]

Lx, Ly = (4, 4)
L = Lx*Ly
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()

perms_v, perms_h = get_perms(Lx, Ly)
perms_extended = [[perms_v[0]]] + [perms_v]*3 + [[perms_v[0]], [perms_h[0]]] +\
                    [perms_h]*3 + [[perms_h[0]], [perms_v[0]]] + [perms_v]*3 + [[perms_v[0]]]
perms_ext_reduced = [perms_v]*3  + [perms_h]*3 + [perms_v]*3
map_ = {i: (i//Ly, i%Lx) for i in range(L)}

peps = qtn.PEPS.rand(Lx, Ly, bond_dim=1, phys_dim=2, cyclic=True)
peps /= peps.norm()
peps_copy = peps.copy()
sv = peps_copy.to_dense()[:, 0]
sv = expm_multiply(-1j * 0.25 * hamil, sv)

for i, V in enumerate(Vlist):
    if i not in control_layers:
        perms = perms_extended[i]
        for perm in perms:
            t = TEBD2D(peps, ham=LocalHam2D(Lx, Ly, 
                    {(map_[perm[2*j]], map_[perm[2*j+1]]): scipy.linalg.logm(V) for j in range(L//2)}, cyclic=True),
                tau=-1, D=4)
            t.sweep(tau=-1)
            peps = t.state
            peps /= peps.norm()

f = quimb.fidelity(peps.to_dense()[:, 0], sv)
print("Fidelity after identity:", f)  # Should be ≈1

Fidelity after identity: 0.9842315725960128


In [92]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import get_perms
sys.path.append("../../src/MPS")
from utils_MPS import (random_mps, apply_localGate, apply_two_site_operator, 
						mps_to_state_vector, get_mps_of_sv, mps_fidelity)
from MPS import trotter, ccU

Lx, Ly = (3, 3)
L= Lx*Ly

# construct Hamiltonian
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=False)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

perms_v, perms_h = get_perms(Lx, Ly)
map_ = {
    0: (0, 0), 1: (1, 0), 2: (2, 0),
    3: (0, 1), 4: (1, 1), 5: (2, 1),
    6: (0, 2), 7: (1, 2), 8: (2, 2)
    
}

In [93]:
map_ = {
    0: (-1, 1), 1: (0, 1), 2: (1, 1),
    3: (-1, 0), 4: (0, 0), 5: (1, 0),
    6: (-1, -1), 7: (0, -1), 8: (1, -1)
    
}

In [94]:
perms_v, perms_h = (
    [[0, 1, 3, 4, 6, 7], [1, 2, 4, 5, 7, 8]],
    [[0, 3, 1, 4, 2, 5], [3, 6, 4, 7, 5, 8]]
)

In [95]:
import scipy
import sys
sys.path.append("../brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms, Z
import rqcopt as oc

SWAP = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])

def trotter(peps, t, L, Lx, Ly, J, g, perms_v, perms_h, dag=False, max_bond_dim=None, dt=0.1, trotter_order=2):
    nsteps = np.abs(int(np.ceil(t/dt)))
    t = t/nsteps
    indices = oc.SplittingMethod.suzuki(2, int(np.log(trotter_order)/np.log(2))).indices
    coeffs = oc.SplittingMethod.suzuki(2, int(np.log(trotter_order)/np.log(2))).coeffs
    
    hloc1 = J*np.kron(Z, Z)
    hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/4
    hlocs = (hloc1, hloc2)
    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))

    for n in range(nsteps):
        for layer, V in enumerate(Vlist_start):
            for perm in perms_h:
                for j in range(len(perm)//2):
                    q0, q1 = (map_[perm[2*j]], map_[perm[2*j+1]])
                    if np.abs(q0[0]-q1[0]) > 1:
                        swapped = False
                        if q0[0]>q1[0]:
                            q0, q1 = (q1, q0)
                            swapped = True
                        q1_org = (q1[0], q1[1])
                        swap_count = 0
                        while q1[0]-q0[0] > 1:
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0]-1, q1[1])),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            swap_count += 1
                            q1 = (q1[0]-1, q1[1])
                        peps.gate_simple_(
                            V,
                            where=(q1, q0) if swapped else (q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                        for i in range(swap_count):
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0]+1, q1[1])),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            q1 = (q1[0]+1, q1[1])
                        assert q1 == q1_org
                    elif np.abs(q0[1]-q1[1]) > 1:
                        swapped = False
                        if q0[1]>q1[1]:
                            q0, q1 = (q1, q0)
                            swapped = True
                        q1_org = (q1[0], q1[1])
                        swap_count = 0
                        while q1[1]-q0[1] > 1:
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0], q1[1]-1)),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            swap_count += 1
                            q1 = (q1[0], q1[1]-1)
                        peps.gate_simple_(
                            V,
                            where=(q1, q0) if swapped else (q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                        for i in range(swap_count):
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0], q1[1]+1)),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            peps /= peps.norm()
                            q1 = (q1[0], q1[1]+1)
                        assert q1 == q1_org
                    else:
                        peps.gate_simple_(
                            V,
                            where=(q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                    peps /= peps.norm()


            for perm in perms_v:
                for j in range(len(perm)//2):
                    q0, q1 = (map_[perm[2*j]], map_[perm[2*j+1]])
                    if np.abs(q0[0]-q1[0]) > 1:
                        swapped = False
                        if q0[0]>q1[0]:
                            q0, q1 = (q1, q0)
                            swapped = True
                        q1_org = (q1[0], q1[1])
                        swap_count = 0
                        while q1[0]-q0[0] > 1:
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0]-1, q1[1])),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            swap_count += 1
                            q1 = (q1[0]-1, q1[1])
                        peps.gate_simple_(
                            V,
                            where=(q1, q0) if swapped else (q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                        for i in range(swap_count):
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0]+1, q1[1])),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            q1 = (q1[0]+1, q1[1])
                        assert q1 == q1_org
                    elif np.abs(q0[1]-q1[1]) > 1:
                        swapped = False
                        if q0[1]>q1[1]:
                            q0, q1 = (q1, q0)
                            swapped = True
                        q1_org = (q1[0], q1[1])
                        swap_count = 0
                        while q1[1]-q0[1] > 1:
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0], q1[1]-1)),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            swap_count += 1
                            q1 = (q1[0], q1[1]-1)
                        peps.gate_simple_(
                            V,
                            where=(q1, q0) if swapped else (q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                        for i in range(swap_count):
                            peps.gate_simple_(
                                SWAP,
                                where=(q1, (q1[0], q1[1]+1)),
                                gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                                max_bond=max_bond_dim,
                                cutoff=1e-19,
                                cutoff_mode='abs',
                            )
                            q1 = (q1[0], q1[1]+1)
                        assert q1 == q1_org
                    else:
                        peps.gate_simple_(
                            V,
                            where=(q0, q1),
                            gauges=peps.gauges if hasattr(peps, 'gauges') else {},
                            max_bond=max_bond_dim,
                            cutoff=1e-19,
                            cutoff_mode='abs',
                        )
                    peps /= peps.norm()
                    
    return peps

t = 0.1
peps = qtn.PEPS.rand(3, 3, bond_dim=2, phys_dim=2, cyclic=True)
peps /= peps.norm()
sv = peps.to_dense()[:, 0]

np.linalg.norm(np.vdot( 
    expm_multiply(-1j * t * hamil, sv)
    , trotter(peps, t, L, Lx, Ly, J, g, perms_v, perms_h, max_bond_dim=4).to_dense()[:, 0]))

0.03851124149530106

In [122]:
t = 0.01
peps = qtn.PEPS.rand(3, 3, bond_dim=4, phys_dim=2, cyclic=True)
peps /= peps.norm()
sv = peps.to_dense()[:, 0]

np.linalg.norm(np.vdot( 
    expm_multiply(-1j * t * hamil, sv)
    , trotter(peps, t, L, Lx, Ly, J, g, perms_v, perms_h, max_bond_dim=4, dt=0.01).to_dense()[:, 0]))

0.8598398896757